Imports des librairies

In [2]:
import pandas as pd

Première analyse des fichiers.

In [4]:
# Dossier contenant les fichiers CSV
dossier = '../data/'

# Liste des années à vérifier
annees = list(range(2005, 2023))

# Liste des types de fichiers à lire
types_fichiers = ['caracteristiques', 'lieux', 'usagers', 'vehicules', 'vehicules-immatricules-baac']

# Initialisation de la liste pour stocker les résultats d'analyse
resultats_analyse = []

# Fonction pour lire les fichiers et extraire les informations
def analyser_fichier(chemin_fichier, type_fichier, annee):
    separateurs = [',', ';', '\t']
    for sep in separateurs:
        try:
            df = pd.read_csv(chemin_fichier, encoding='ISO-8859-1', delimiter=sep, on_bad_lines='skip', low_memory=False)
            if len(df.columns) > 1:
                stats_colonnes = {}
                for col in df.columns:
                    stats_colonnes[col] = {
                        'type': df[col].dtype,
                        'taux_nan': df[col].isna().mean() * 100,
                        'mode': df[col].mode().iloc[0] if not df[col].mode().empty else None,
                        'min': df[col].min() if df[col].dtype in ['float64', 'int64'] else None,
                        'max': df[col].max() if df[col].dtype in ['float64', 'int64'] else None,
                        'moyenne': df[col].mean() if df[col].dtype in ['float64', 'int64'] else None,
                        'mediane': df[col].median() if df[col].dtype in ['float64', 'int64'] else None,
                        'ecart_type': df[col].std() if df[col].dtype in ['float64', 'int64'] else None
                    }
                return {
                    'nom_fichier': f'{type_fichier}_{annee}.csv',
                    'chemin_fichier_complet': chemin_fichier,
                    'nombre_colonnes': len(df.columns),
                    'nombre_lignes': len(df),
                    'colonnes': df.columns.tolist(),
                    'statut': 'Succès',
                    'separateur': sep,
                    'stats_colonnes': stats_colonnes
                }
        except Exception:
            continue
    return {
        'nom_fichier': f'{type_fichier}_{annee}.csv',
        'chemin_fichier_complet': chemin_fichier,
        'nombre_colonnes': 1,
        'nombre_lignes': 0,
        'colonnes': [],
        'statut': 'Échec de lecture',
        'separateur': 'N/A',
        'stats_colonnes': {}
    }

# Analyse des fichiers et stockage des résultats
for type_fichier in types_fichiers:
    for annee in annees:
        chemin_fichier = f'{dossier}{type_fichier}_{annee}.csv'
        resultat = analyser_fichier(chemin_fichier, type_fichier, annee)
        resultat['type_fichier'] = type_fichier
        resultat['annee'] = annee
        resultats_analyse.append(resultat)

# Création d'un DataFrame pour les résultats d'analyse
analyse_df = pd.DataFrame(resultats_analyse)

# Extraction et structuration des statistiques de colonnes
stats_data = []
for resultat in resultats_analyse:
    for col, stats in resultat['stats_colonnes'].items():
        stats_data.append({
            'nom_fichier': resultat['nom_fichier'],
            'type_fichier': resultat['type_fichier'],
            'annee': resultat['annee'],
            'colonne': col,
            **stats
        })

# Création d'un DataFrame pour les statistiques de colonnes
stats_df = pd.DataFrame(stats_data)

# Export des résultats d'analyse et des statistiques en fichier Excel
with pd.ExcelWriter('resultats_analyse.xlsx') as writer:
    analyse_df.to_excel(writer, sheet_name='Résultats Analyse', index=False)
    stats_df.to_excel(writer, sheet_name='Stats Colonnes', index=False)

print("Export terminé.")


Export terminé.


In [5]:
analyse_df.head()

,nom_fichier,chemin_fichier_complet,nombre_colonnes,nombre_lignes,colonnes,statut,separateur,stats_colonnes,type_fichier,annee
0,caracteristiques_2005.csv,../data/caracteristiques_2005.csv,16,87026,"[Num_Acc, an, mois, jour, hrmn, lum, agg, int,...",Succès,",","{'Num_Acc': {'type': int64, 'taux_nan': 0.0, '...",caracteristiques,2005
1,caracteristiques_2006.csv,../data/caracteristiques_2006.csv,16,82993,"[Num_Acc, an, mois, jour, hrmn, lum, agg, int,...",Succès,",","{'Num_Acc': {'type': int64, 'taux_nan': 0.0, '...",caracteristiques,2006
2,caracteristiques_2007.csv,../data/caracteristiques_2007.csv,16,83850,"[Num_Acc, an, mois, jour, hrmn, lum, agg, int,...",Succès,",","{'Num_Acc': {'type': int64, 'taux_nan': 0.0, '...",caracteristiques,2007
3,caracteristiques_2008.csv,../data/caracteristiques_2008.csv,16,76767,"[Num_Acc, an, mois, jour, hrmn, lum, agg, int,...",Succès,",","{'Num_Acc': {'type': int64, 'taux_nan': 0.0, '...",caracteristiques,2008
4,caracteristiques_2009.csv,../data/caracteristiques_2009.csv,16,74409,"[Num_Acc, an, mois, jour, hrmn, lum, agg, int,...",Succès,\t,"{'Num_Acc': {'type': int64, 'taux_nan': 0.0, '...",caracteristiques,2009


Analyse des colonnes

In [7]:
# Initialisation du DataFrame final
informations_colonnes = []

# Filtrer le DataFrame pour ne conserver que les lignes où le statut est "Succès"
analyse_df_succes = analyse_df[analyse_df['statut'] == 'Succès']

# Parcourir chaque fichier dans le DataFrame filtré
for idx, row in analyse_df_succes.iterrows():
    chemin_fichier = row['chemin_fichier_complet']
    type_fichier = row['type_fichier']
    separateur = row['separateur']
    
    # Charger le fichier CSV
    try:
        df = pd.read_csv(chemin_fichier, sep=separateur, encoding='ISO-8859-1', low_memory=False)
        
        # Obtenir les informations sur les colonnes
        for col in df.columns:
            info_col = {
                'chemin_fichier_complet': chemin_fichier,
                'type_fichier': type_fichier,
                'nom_colonne': col,
                'nombre_lignes': len(df),
                'format': df[col].dtype
            }
            informations_colonnes.append(info_col)
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier {chemin_fichier} : {e}")

# Créer le DataFrame final
analyse_colonnes_df = pd.DataFrame(informations_colonnes)

analyse_colonnes_df.head()


,chemin_fichier_complet,type_fichier,nom_colonne,nombre_lignes,format
0,../data/caracteristiques_2005.csv,caracteristiques,Num_Acc,87026,int64
1,../data/caracteristiques_2005.csv,caracteristiques,an,87026,int64
2,../data/caracteristiques_2005.csv,caracteristiques,mois,87026,int64
3,../data/caracteristiques_2005.csv,caracteristiques,jour,87026,int64
4,../data/caracteristiques_2005.csv,caracteristiques,hrmn,87026,int64


In [8]:
analyse_colonnes_df.columns

Index(['chemin_fichier_complet', 'type_fichier', 'nom_colonne',
       'nombre_lignes', 'format'],
      dtype='object')

DF Fusion_colonne par type de fichier avec somme sur nb de ligne et aggrégation sur format

In [10]:
fusion_colonne = analyse_colonnes_df.groupby(['type_fichier', 'nom_colonne'], as_index=False).agg(
    nombre_lignes=('nombre_lignes', 'sum'),
    format=('format', lambda x: ','.join(sorted(set(str(i) for i in x)))),  # Convertir les valeurs en chaînes de caractères
)


In [11]:
fusion_colonne.head()

,type_fichier,nom_colonne,nombre_lignes,format
0,caracteristiques,Accident_Id,55302,int64
1,caracteristiques,Num_Acc,1121571,int64
2,caracteristiques,adr,1176873,object
3,caracteristiques,agg,1176873,int64
4,caracteristiques,an,1176873,int64


In [12]:
fusion_colonne[fusion_colonne['type_fichier'] == 'caracteristiques']

,type_fichier,nom_colonne,nombre_lignes,format
0,caracteristiques,Accident_Id,55302,int64
1,caracteristiques,Num_Acc,1121571,int64
2,caracteristiques,adr,1176873,object
3,caracteristiques,agg,1176873,int64
4,caracteristiques,an,1176873,int64
5,caracteristiques,atm,1176873,"float64,int64"
6,caracteristiques,col,1176873,"float64,int64"
7,caracteristiques,com,1176873,"float64,int64,object"
8,caracteristiques,dep,1176873,"int64,object"
9,caracteristiques,gps,958469,object


Attention Accident_id et Num_Acc à fusioner, attention aux formats différents

In [14]:
fusion_colonne[fusion_colonne['type_fichier'] == 'lieux']

,type_fichier,nom_colonne,nombre_lignes,format
17,lieux,Num_Acc,1176873,int64
18,lieux,catr,1176873,"float64,int64"
19,lieux,circ,1176873,"float64,int64"
20,lieux,env1,958469,"float64,int64"
21,lieux,infra,1176873,"float64,int64"
22,lieux,larrout,1176873,"float64,int64,object"
23,lieux,lartpc,1176873,"float64,int64,object"
24,lieux,nbv,1176873,"float64,int64,object"
25,lieux,plan,1176873,"float64,int64"
26,lieux,pr,1176873,"float64,object"


In [15]:
fusion_colonne[fusion_colonne['type_fichier'] == 'usagers']

,type_fichier,nom_colonne,nombre_lignes,format
36,usagers,Num_Acc,2636377,int64
37,usagers,actp,2636377,"float64,int64,object"
38,usagers,an_nais,2636377,"float64,int64"
39,usagers,catu,2636377,int64
40,usagers,etatp,2636377,"float64,int64"
41,usagers,grav,2636377,int64
42,usagers,id_usager,255910,object
43,usagers,id_vehicule,494182,object
44,usagers,locp,2636377,"float64,int64"
45,usagers,num_veh,2636377,object


In [16]:
fusion_colonne[fusion_colonne['type_fichier'] == 'vehicules']

,type_fichier,nom_colonne,nombre_lignes,format
53,vehicules,Num_Acc,2009395,int64
54,vehicules,catv,2009395,int64
55,vehicules,choc,2009395,"float64,int64"
56,vehicules,id_vehicule,373584,object
57,vehicules,manv,2009395,"float64,int64"
58,vehicules,motor,373584,int64
59,vehicules,num_veh,2009395,object
60,vehicules,obs,2009395,"float64,int64"
61,vehicules,obsm,2009395,"float64,int64"
62,vehicules,occutc,2009395,"float64,int64"


Version 2 analyse en vue de concat et fusion

In [18]:
import pandas as pd

# Dossier contenant les fichiers CSV
dossier = '../data/'

# Liste des années à vérifier
annees = list(range(2005, 2023))

# Liste des types de fichiers à lire
types_fichiers = ['caracteristiques', 'lieux', 'usagers', 'vehicules', 'vehicules-immatricules-baac']

# Initialisation de la liste pour stocker les résultats d'analyse
resultats_analyse = []

# Dictionnaire pour stocker les DataFrames
dataframes = {}

# Fonction pour lire les fichiers et extraire les informations
def analyser_fichier(chemin_fichier, type_fichier, annee):
    separateurs = [',', ';', '\t']
    for sep in separateurs:
        try:
            df = pd.read_csv(chemin_fichier, encoding='ISO-8859-1', delimiter=sep, on_bad_lines='skip', low_memory=False)
            dataframes[f'{type_fichier}_{annee}'] = df  # Stocke le DataFrame dans le dictionnaire
            if len(df.columns) > 1:
                stats_colonnes = {}
                for col in df.columns:
                    stats_colonnes[col] = {
                        'type': df[col].dtype,
                        'taux_nan': df[col].isna().mean() * 100,
                        'mode': df[col].mode().iloc[0] if not df[col].mode().empty else None,
                        'min': df[col].min() if df[col].dtype in ['float64', 'int64'] else None,
                        'max': df[col].max() if df[col].dtype in ['float64', 'int64'] else None,
                        'moyenne': df[col].mean() if df[col].dtype in ['float64', 'int64'] else None,
                        'mediane': df[col].median() if df[col].dtype in ['float64', 'int64'] else None,
                        'ecart_type': df[col].std() if df[col].dtype in ['float64', 'int64'] else None
                    }
                return {
                    'nom_fichier': f'{type_fichier}_{annee}.csv',
                    'chemin_fichier_complet': chemin_fichier,
                    'nombre_colonnes': len(df.columns),
                    'nombre_lignes': len(df),
                    'colonnes': df.columns.tolist(),
                    'statut': 'Succès',
                    'separateur': sep,
                    'stats_colonnes': stats_colonnes
                }
        except Exception:
            continue
    return {
        'nom_fichier': f'{type_fichier}_{annee}.csv',
        'chemin_fichier_complet': chemin_fichier,
        'nombre_colonnes': 1,
        'nombre_lignes': 0,
        'colonnes': [],
        'statut': 'Échec de lecture',
        'separateur': 'N/A',
        'stats_colonnes': {}
    }

# Analyse des fichiers et stockage des résultats
for type_fichier in types_fichiers:
    for annee in annees:
        chemin_fichier = f'{dossier}{type_fichier}_{annee}.csv'
        resultat = analyser_fichier(chemin_fichier, type_fichier, annee)
        resultat['type_fichier'] = type_fichier
        resultat['annee'] = annee
        resultats_analyse.append(resultat)

# Création d'un DataFrame pour les résultats d'analyse
analyse_df = pd.DataFrame(resultats_analyse)

# Extraction et structuration des statistiques de colonnes
stats_data = []
for resultat in resultats_analyse:
    for col, stats in resultat['stats_colonnes'].items():
        stats_data.append({
            'nom_fichier': resultat['nom_fichier'],
            'type_fichier': resultat['type_fichier'],
            'annee': resultat['annee'],
            'colonne': col,
            **stats
        })

# Création d'un DataFrame pour les statistiques de colonnes
stats_df = pd.DataFrame(stats_data)

# Export des résultats d'analyse et des statistiques en fichier Excel
with pd.ExcelWriter('resultats_analyse.xlsx') as writer:
    analyse_df.to_excel(writer, sheet_name='Résultats Analyse', index=False)
    stats_df.to_excel(writer, sheet_name='Stats Colonnes', index=False)

print("Export terminé.")



Export terminé.


Concaténation

In [59]:


# Pour chaque type de fichier
for type_fichier in types_fichiers:
    # Initialiser une liste vide pour stocker les dataframes
    dfs = []

    # Parcourir les clés du dictionnaire
    for key in dataframes_concat.keys():
        # Vérifier si la clé commence par le type de fichier
        if key.startswith(type_fichier):
            # Si c'est le cas, ajouter le dataframe à la liste
            dfs.append(dataframes_concat[key])

    # Concaténer tous les dataframes de la liste en un seul
    # et stocker le résultat dans un nouveau dataframe
    globals()[f'df_concat_{type_fichier}'] = pd.concat(dfs)





In [61]:
df_concat_caracteristiques.head()

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep,Accident_Id
0,2.005000e+11,5,1,12,1900,3,2,1,1.0,3.0,11.0,CD41B,M,5051500.0,294400.0,590,NaN
1,2.005000e+11,5,1,21,1600,1,2,1,1.0,1.0,51.0,rue de Lille,M,5053700.0,280200.0,590,NaN
2,2.005000e+11,5,1,21,1845,3,1,1,2.0,1.0,51.0,NaN,M,5054600.0,280000.0,590,NaN
3,2.005000e+11,5,1,4,1615,1,1,1,1.0,5.0,82.0,NaN,M,5098700.0,240800.0,590,NaN
4,2.005000e+11,5,1,10,1945,3,1,1,3.0,6.0,478.0,NaN,M,5096400.0,247500.0,590,NaN


In [63]:
df_concat_lieux.head()

,Num_Acc,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,prof,plan,lartpc,larrout,surf,infra,situ,env1,vma
0,200500000001,3.0,41.0,0.0,B,2.0,2,1.0,430.0,0.0,1.0,1.0,0,63,1.0,0.0,1.0,0.0,NaN
1,200500000002,2.0,41.0,0.0,NaN,0.0,2,0.0,0.0,1.0,1.0,1.0,0,100,1.0,0.0,5.0,0.0,NaN
2,200500000003,2.0,41.0,0.0,NaN,0.0,0,0.0,0.0,1.0,1.0,1.0,0,0,2.0,0.0,5.0,0.0,NaN
3,200500000004,3.0,916.0,0.0,NaN,2.0,2,0.0,0.0,0.0,1.0,1.0,0,0,1.0,0.0,1.0,0.0,NaN
4,200500000005,3.0,110.0,0.0,NaN,2.0,2,24.0,630.0,0.0,1.0,3.0,0,59,2.0,0.0,3.0,0.0,NaN


In [65]:
df_concat_usagers.head()

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,an_nais,num_veh,id_vehicule,secu1,secu2,secu3,id_usager
0,200500000001,1.0,1,4,1,1.0,11.0,0.0,0,0.0,1976.0,A01,NaN,NaN,NaN,NaN,NaN
1,200500000001,1.0,1,3,2,3.0,11.0,0.0,0,0.0,1968.0,B02,NaN,NaN,NaN,NaN,NaN
2,200500000001,2.0,2,1,1,0.0,11.0,0.0,0,0.0,1964.0,B02,NaN,NaN,NaN,NaN,NaN
3,200500000001,4.0,2,1,1,0.0,31.0,0.0,0,0.0,2004.0,B02,NaN,NaN,NaN,NaN,NaN
4,200500000001,5.0,2,1,1,0.0,11.0,0.0,0,0.0,1998.0,B02,NaN,NaN,NaN,NaN,NaN


In [67]:
df_concat_vehicules.head()

,Num_Acc,senc,catv,occutc,obs,obsm,choc,manv,num_veh,id_vehicule,motor,Id_accident,Lettre Conventionnelle VÃ©hicule,AnnÃ©e,Lieu Admin Actuel - Territoire Nom,Type Accident - LibellÃ© (old),CNIT,CatÃ©gorie vÃ©hicule,Age vÃ©hicule,Type Accident - LibellÃ©
0,2.005000e+11,0.0,7.0,0.0,0.0,2.0,1.0,1.0,A01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.005000e+11,0.0,7.0,0.0,0.0,2.0,8.0,10.0,B02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.005000e+11,0.0,7.0,0.0,0.0,2.0,7.0,16.0,A01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.005000e+11,0.0,2.0,0.0,0.0,2.0,1.0,1.0,B02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.005000e+11,0.0,2.0,0.0,0.0,2.0,1.0,1.0,A01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Fusion acc_id et Num_acc